In [1]:
import time
import numpy as np
import pandas as pd
import time
import pickle
import sys
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https:/1xbet.whoscored.com/')
driver.implicitly_wait(3)
driver.close()

In [23]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https:/1xbet.whoscored.com/')
#driver.implicitly_wait(3)
driver.close()

In [5]:
def crawling_league_teams(region, tournaments, api_delay_term=5):

    """
    cawling league team_id and team name datas
    
    Arg :
        region : region id (from whoscored) for certain league ex) england --- 252, spain --- 206
        tournaments : tournament(or league) id ex) premier league --- 2,
    
    return :
        crawling league team_id, team_name datas belong team_id parameter
        return pandas dataframe columns=team_id, team_name
    

    """
    
    # connect webdriver
    url = "https://1xbet.whoscored.com/Regions/" + str(region) + "/Tournaments/" + str(tournaments)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # wait get league team datas
    time.sleep(api_delay_term) 
    
    # make pandas dataframe
    team_df = pd.DataFrame(columns=["team_id","team_name"])

    # get team data
    teams = driver.find_element(By.XPATH, "//*[@id='standings-20934-content']")
    teams = teams.find_elements(By.CSS_SELECTOR, "a.team-link")
    
    for team in teams:
        team_name = team.get_attribute("innerHTML")
        team_id = team.get_attribute("href").split("/")[4]
        team_df.loc[len(team_df)] = {"team_id":team_id, "team_name":team_name }
        
    # close webdriver
    driver.close()
    
    return team_df

- Premier League teams and id.

In [6]:
PL_df = crawling_league_teams(252, 2, api_delay_term=3)
PL_df

,team_id,team_name
0,13,Arsenal
1,167,Manchester City
2,23,Newcastle
3,32,Manchester United
4,26,Liverpool
5,30,Tottenham
6,24,Aston Villa
7,211,Brighton
8,189,Brentford
9,170,Fulham


In [7]:
def crawling_player_summary(team_id, api_delay_term=5):
    """
    crawling player statistics of certain team
    
    Args :
        team_id : team number 
        
    return :
        player statistics (dataframe)
    
    """

    # connect webdriver
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    # wait for getting data
    time.sleep(api_delay_term)
    
    # make pandas dataframe
    player_summary_df = pd.DataFrame(columns=[
        "player_number", "name","age","position","tall","weight","games",
        "mins", "goals", "asists", "yel", "red", "spg", "ps","aw","motm", "rating"
        ])
    
    # get player summay datas
    elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")
    for element in elements:
        
        player_dict = {
            "player_number": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, ("a"))[0].get_attribute("href").split("/")[4], 
            "name": element.find_element(By.CSS_SELECTOR, "td > a > span").get_attribute("textContent").split("\t")[0],     
            "age": element.find_element(By.CSS_SELECTOR, "td > span > span:nth-child(1)").get_attribute("textContent").split("\t")[0],
            "position": element.find_element(By.CSS_SELECTOR, "td > span > span:nth-child(2)").get_attribute("textContent").split("\t")[0][2:], 
            "tall": element.find_elements(By.CSS_SELECTOR, ("td"))[2].get_attribute("textContent").split("\t")[0],
            "weight": element.find_elements(By.CSS_SELECTOR, ("td"))[3].get_attribute("textContent").split("\t")[0], 
            "games": element.find_elements(By.CSS_SELECTOR, ("td"))[4].get_attribute("textContent").split("\t")[0], 
            "mins": element.find_elements(By.CSS_SELECTOR, ("td"))[5].get_attribute("textContent").split("\t")[0],
            "goals": element.find_elements(By.CSS_SELECTOR, ("td"))[6].get_attribute("textContent").split("\t")[0],
            "asists": element.find_elements(By.CSS_SELECTOR, ("td"))[7].get_attribute("textContent").split("\t")[0],
            "yel": element.find_elements(By.CSS_SELECTOR, ("td"))[8].get_attribute("textContent").split("\t")[0],
            "red": element.find_elements(By.CSS_SELECTOR, ("td"))[9].get_attribute("textContent").split("\t")[0],
            "spg": element.find_elements(By.CSS_SELECTOR, ("td"))[10].get_attribute("textContent").split("\t")[0],
            "ps": element.find_elements(By.CSS_SELECTOR, ("td"))[11].get_attribute("textContent").split("\t")[0],
            "aw": element.find_elements(By.CSS_SELECTOR, ("td"))[12].get_attribute("textContent").split("\t")[0],
            "motm": element.find_elements(By.CSS_SELECTOR, ("td"))[13].get_attribute("textContent").split("\t")[0],
            "rating": element.find_elements(By.CSS_SELECTOR, ("td"))[14].get_attribute("textContent").split("\t")[0],
        }
        
        player_summary_df.loc[len(player_summary_df)] = player_dict
        
    # close webdriver
    driver.close()
    player_summary_df.replace("-", "0", inplace=True)
    return player_summary_df

- Player stat table (Liverpool) 

In [8]:
player_liverpool = crawling_player_summary(26, api_delay_term=5)

In [9]:
player_liverpool

,player_number,name,age,position,tall,weight,games,mins,goals,asists,yel,red,spg,ps,aw,motm,rating
0,377168,Luis Díaz,26,AM(L),178,65,8(4),727,4,2,2,0,1.6,83.3,0.5,2,7.10
1,352825,Cody Gakpo,23,"AM(CL),FW",193,76,14(2),1138,6,2,0,0,1.9,77.9,1.4,0,7.08
2,108226,Mohamed Salah,30,"AM(CLR),FW",175,71,32(1),2855,17,7,2,0,3.2,78.6,0.2,3,7.07
3,318871,Trent Alexander-Arnold,24,"D(R),M(R)",175,72,29(3),2482,1,8,4,0,0.9,78.9,0.5,1,7.01
4,95408,Virgil van Dijk,31,D(C),195,92,28,2476,3,0,2,0,1,90.3,2.9,1,6.91
5,114147,Alisson,30,GK,193,91,33,2970,0,1,0,0,0,83.4,0.3,2,6.88
6,96182,Roberto Firmino,31,"M(CLR),FW",181,76,12(11),1138,9,4,0,0,1.6,80.2,0.6,2,6.84
7,115726,Andrew Robertson,29,"D(L),M(L)",178,64,26(4),2320,0,8,3,0,0.4,84.1,0.6,0,6.82
8,400828,Darwin Núñez,23,"AM(CL),FW",187,81,17(10),1553,9,3,1,1,3,67.3,1.3,2,6.82
9,74939,Thiago Alcântara,32,M(C),174,70,14(4),1256,0,0,2,0,0.6,86.5,0.8,1,6.82


In [3]:
# Premier league 1920 plyaer stat table(URL)
URL = 'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/7466/Stages/16546/PlayerStatistics/Spain-LaLiga-2018-2019'

In [4]:
def crawling_player_statistics(URL):
    """
    crawling player statistics of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)    
    
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()
    driver.find_element(By.XPATH, '//*[@id="apps"]/dd[2]/a').click()
    time.sleep(5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-summary > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\\")[0]
    page = int(page.split('/')[1].split(' ')[0])
    player_statistics_df = pd.DataFrame(columns=[
        "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
        "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
      
        for i in range(page):
            if i != 0:
                driver.find_element(By.ID, 'next').click()
                time.sleep(3)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
                "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, ("a"))[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Yel": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Red": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "SpG": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "PS%": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "AerialsWon": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "MoM": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict

            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
            
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)
    

In [5]:
PL_player_statistics_2223 = crawling_player_statistics(URL)

page 1:   2%|▏         | 1/55 [00:08<07:41,  8.55s/it]


KeyboardInterrupt: 

In [43]:
PL_player_statistics_2223.head()

,player_name,team_number,team_name,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MoM,Rating
0,Asmir Begovic,31,Everton,1,90,0,0,0,0,0,64,1,1,7.82
1,Kevin De Bruyne,167,Man City,26(3),2266,7,16,1,0,2.2,80.1,0.4,9,7.67
2,Erling Haaland,167,Man City,29(1),2415,34,7,5,0,3.5,74.9,1.4,6,7.62
3,Kieran Trippier,23,Newcastle,33,2898,1,6,5,0,0.4,75.4,1.4,8,7.48
4,Harry Kane,30,Tottenham,34,3048,25,3,5,0,3.5,72.4,1.7,7,7.48


In [164]:
#PL_player_statistics_2223.to_csv('PL_player_summary_2223.csv')
    time.sleep(2.5)

    driver.find_element(By.ID, 'next').click()

    time.sleep(2.5)

    driver.find_element(By.ID, 'first').click()

In [12]:
def crawling_player_statistics_defensive(URL):
    """
    crawling player statistics(defensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(2.5) 
    
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()

    time.sleep(2.5)
    
    driver.find_element(By.LINK_TEXT, "Defensive").click()

    
    time.sleep(2.5)

    
    driver.find_element(By.XPATH, '/html/body/div[5]/div[5]/div[3]/div[1]/div[2]/dl[2]/dd[2]/a').click()


    time.sleep(2.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-defensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
            "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(2.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
                "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Tackles": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Inter": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Fouls": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Offsides": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Clear": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Drb": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Blocks": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "OwnG": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)
    

In [38]:
PL_player_defensive_2223 = crawling_player_statistics_defensive(URL)

  0%|          | 0/56 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [167]:
PL_player_defensive_2223.head()

,player_name,team_number,team_name,age,position,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,Erling Haaland,167,Man City,22,FW,15(1),1311,0.1,0.1,0.8,0,0.4,0.1,0.1,0,7.79
1,Kieran Trippier,23,Newcastle,32,"D(LR),M(R)",18,1597,2.2,1.4,1,0.3,1.7,1,0.6,0,7.70
2,Kevin De Bruyne,167,Man City,31,"M(CLR),FW",16(1),1405,1.1,0.2,0.9,0,0.4,0.8,0.1,0,7.68
3,Gabriel Jesus,13,Arsenal,25,"AM(LR),FW",14,1226,1.4,0.6,1.7,0,0.6,0.6,0.1,0,7.59
4,Harry Kane,30,Tottenham,29,"AM(C),FW",18,1608,0.3,0.1,0.7,0.1,0.5,0.2,0.2,0,7.55


In [168]:
#PL_player_defensive_2223.to_csv('PL_player_defensive_2223.csv') Offensive

In [13]:
def crawling_player_statistics_offensive(URL):
    """
    crawling player statistics(offensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(2.5) 
    
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()

    time.sleep(2.5)
    
    driver.find_element(By.LINK_TEXT, "Offensive").click()

    
    time.sleep(2.5)

    
    driver.find_element(By.LINK_TEXT, "All players").click()
    
    time.sleep(2.5)

    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-offensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\t")[0]
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
            "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(2.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
                "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "SpG": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "KeyP": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Dribble": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Fouled": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Off": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Disp": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "UnsTch": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[13].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()

    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)

In [47]:
PL_player_offensive_2223 = crawling_player_statistics_offensive(URL)

page 3:   5%|▌         | 3/56 [00:23<06:53,  7.80s/it]


In [171]:
PL_player_offensive_2223.head()

,player_name,team_number,team_name,age,position,Apps,Mins,Goals,Assists,SpG,KeyP,Dribble,Fouled,Off,Disp,UnsTch,Rating
0,Erling Haaland,167,Man City,22,FW,15(1),1311,21,3,4.1,1,0.3,0.7,0.3,0.9,1.5,7.79
1,Kieran Trippier,23,Newcastle,32,"D(LR),M(R)",18,1597,1,4,0.4,2.6,0.3,0.7,0.2,0.1,0.6,7.70
2,Kevin De Bruyne,167,Man City,31,"M(CLR),FW",16(1),1405,3,9,2.6,3.5,0.8,0.8,0.2,1.2,1.5,7.68
3,Gabriel Jesus,13,Arsenal,25,"AM(LR),FW",14,1226,5,5,3.6,1.6,1.9,2.7,0.6,2.9,3.9,7.59
4,Harry Kane,30,Tottenham,29,"AM(C),FW",18,1608,15,1,3.6,1.7,0.8,1.4,0.3,2,1.7,7.55


In [179]:
#PL_player_offensive_2223.to_csv('PL_player_offensive_2223.csv')

In [14]:
def crawling_player_statistics_passing(URL):

    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(2.5) 
    
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()

    time.sleep(2.5)
    
    driver.find_element(By.LINK_TEXT, "Passing").click()

    
    time.sleep(2.5)

    
    driver.find_element(By.LINK_TEXT, "All players").click()


    time.sleep(2.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-passing > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Assists",
            "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(2.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins","Assists",
                "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "KeyP": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "AvgP": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "PS%": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Crosses": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "LongB": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "ThrB": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)

    return(player_statistics_df)

In [9]:
PL_player_passing_2223 = crawling_player_statistics_passing(URL)

page 7:  13%|█▎        | 7/55 [00:54<06:14,  7.81s/it]


KeyboardInterrupt: 

In [175]:
PL_player_passing_2223.head()

,player_name,team_number,team_name,age,position,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,Erling Haaland,167,Man City,22,FW,15(1),1311,3,1,14.1,75.7,0,0.1,0.1,7.79
1,Kieran Trippier,23,Newcastle,32,"D(LR),M(R)",18,1597,4,2.6,50.2,75.9,3.7,3.5,0.3,7.70
2,Kevin De Bruyne,167,Man City,31,"M(CLR),FW",16(1),1405,9,3.5,44.9,79.9,2.4,2.9,0.5,7.68
3,Gabriel Jesus,13,Arsenal,25,"AM(LR),FW",14,1226,5,1.6,27.6,81.9,0.1,0.5,0.1,7.59
4,Harry Kane,30,Tottenham,29,"AM(C),FW",18,1608,1,1.7,22.4,71.7,0.2,1.7,0.1,7.55


In [178]:
PL_player_passing_2223.to_csv('PL_player_passing_2223.csv')

In [180]:
passing=pd.read_csv('PL_player_passing_2223.csv')
summary=pd.read_csv('PL_player_summary_2223.csv')
offensive=pd.read_csv('PL_player_offensive_2223.csv')
defensive=pd.read_csv('PL_player_defensive_2223.csv')

In [181]:
PL_player_2223 = pd.merge(pd.merge(passing,summary), pd.merge(offensive,defensive))

In [182]:
PL_player_2223.head()

,Unnamed: 0,player_name,team_number,team_name,age,position,Apps,Mins,Assists,KeyP,...,Disp,UnsTch,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG
0,0,Erling Haaland,167,Man City,22,FW,15(1),1311,3,1.0,...,0.9,1.5,0.1,0.1,0.8,0.0,0.4,0.1,0.1,0
1,1,Kieran Trippier,23,Newcastle,32,"D(LR),M(R)",18,1597,4,2.6,...,0.1,0.6,2.2,1.4,1.0,0.3,1.7,1.0,0.6,0
2,2,Kevin De Bruyne,167,Man City,31,"M(CLR),FW",16(1),1405,9,3.5,...,1.2,1.5,1.1,0.2,0.9,0.0,0.4,0.8,0.1,0
3,3,Gabriel Jesus,13,Arsenal,25,"AM(LR),FW",14,1226,5,1.6,...,2.9,3.9,1.4,0.6,1.7,0.0,0.6,0.6,0.1,0
4,4,Harry Kane,30,Tottenham,29,"AM(C),FW",18,1608,1,1.7,...,2.0,1.7,0.3,0.1,0.7,0.1,0.5,0.2,0.2,0


# crwaling player stats at once

In [15]:
def crwaling_player_stats_at_once(URL):
    stats_summary = crawling_player_statistics(URL)
    print('player stats summary : done')
    stats_defensive = crawling_player_statistics_defensive(URL)
    print('player stats defensive : done')
    stats_offensive = crawling_player_statistics_offensive(URL)
    print('player stats offensive : done')
    stats_passing = crawling_player_statistics_passing(URL)
    print('player stats pasing : done')
    player_stats = pd.merge(pd.merge(stats_passing, stats_summary), 
                            pd.merge(stats_offensive, stats_defensive))
    return(player_stats)

In [7]:
# PL2022-23
URL = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023'

In [16]:
PL_2223 = crwaling_player_stats_at_once(URL)

page 55: 100%|██████████| 55/55 [08:05<00:00,  8.82s/it]
player stats summary : done
page 55: 100%|██████████| 55/55 [09:11<00:00, 10.03s/it]
player stats defensive : done
page 22:  40%|████      | 22/55 [03:24<06:40, 12.14s/it]

In [152]:
PL_2223.head()

,AvgP,PS%,Crosses,LongB,ThrB,Yel,Red,AerialsWon,MoM,player_name,...,UnsTch,Rating,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG


In [36]:
PL_2223.to_csv('PL_player_2223.csv')

In [12]:
URL1516 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/5826/Stages/12496/PlayerStatistics/England-Premier-League-2015-2016'
URL1617 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/6335/Stages/13796/PlayerStatistics/England-Premier-League-2016-2017'
URL1718 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/6829/Stages/15151/PlayerStatistics/England-Premier-League-2017-2018'
URL1819 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7361/Stages/16368/PlayerStatistics/England-Premier-League-2018-2019'

In [42]:
PL_1516 = crwaling_player_stats_at_once(URL1516)
PL_1617 = crwaling_player_stats_at_once(URL1617)
PL_1718 = crwaling_player_stats_at_once(URL1718)
PL_1819 = crwaling_player_stats_at_once(URL1819)

page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [02:33<00:00,  5.10s/it]
player stats summary : done
page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [02:40<00:00,  5.35s/it]
player stats defensive : done
page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [02:54<00:00,  5.83s/it]
player stats offensive : done
page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [02:33<00:00,  5.10s/it]
player stats pasing : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:28<00:00,  5.13s/it]
player stats summary : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:35<00:00,  5.35s/it]
player stats defensive : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:43<00:00

In [13]:
PL_1718 = crwaling_player_stats_at_once(URL1718)

page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:29<00:00,  5.17s/it]
player stats summary : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:36<00:00,  5.40s/it]
player stats defensive : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:38<00:00,  5.48s/it]
player stats offensive : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:28<00:00,  5.11s/it]
player stats pasing : done


In [ ]:
PL_1718.to_csv('PL_player_1718.csv')

In [44]:
PL_1516.to_csv('PL_player_1516.csv')
PL_1617.to_csv('PL_player_1617.csv')
PL_1718.to_csv('PL_player_1718.csv')
PL_1819.to_csv('PL_player_1819.csv')